In [1]:
!pip install git+https://github.com/d2l-ai/d2l-en/

  Cloning https://github.com/d2l-ai/d2l-en/ to /tmp/pip-req-build-18o9ecqy
  Running command git clone -q https://github.com/d2l-ai/d2l-en/ /tmp/pip-req-build-18o9ecqy
  Created wheel for d2l: filename=d2l-0.15.1-cp36-none-any.whl size=63577 sha256=86215dcad4d9ee724d098d9b325fccf2e3ea2dd5e4f8f98288c7264114d06b1d
  Stored in directory: /tmp/pip-ephem-wheel-cache-ay627w1i/wheels/b2/77/83/e796dce4511b13d56264bc7c3703feb8a71ec12fdb0401cabc
Successfully built d2l


In [2]:
import torch
from torch import nn
from d2l import torch as d2l

In [3]:
def trans_conv(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] + h - 1, X.shape[1] + w - 1))
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            Y[i: i + h, j: j + w] += X[i, j] * K
    return Y

In [4]:
X = torch.tensor([[0., 1], [2, 3]])
K = torch.tensor([[0., 1], [2, 3]])
trans_conv(X, K)

tensor([[ 0.,  0.,  1.],
        [ 0.,  4.,  6.],
        [ 4., 12.,  9.]])

In [5]:
X, K = X.reshape(1, 1, 2, 2), K.reshape(1, 1, 2, 2)
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[ 0.,  0.,  1.],
          [ 0.,  4.,  6.],
          [ 4., 12.,  9.]]]], grad_fn=<SlowConvTranspose2DBackward>)

In [6]:
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, padding=1, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[4.]]]], grad_fn=<SlowConvTranspose2DBackward>)

In [7]:
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, stride=2, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[0., 0., 0., 1.],
          [0., 0., 2., 3.],
          [0., 2., 0., 3.],
          [4., 6., 6., 9.]]]], grad_fn=<SlowConvTranspose2DBackward>)

In [8]:
X = torch.rand(size=(1, 10, 16, 16))
conv = nn.Conv2d(10, 20, kernel_size=5, padding=2, stride=3)
tconv = nn.ConvTranspose2d(20, 10, kernel_size=5, padding=2, stride=3)
tconv(conv(X)).shape == X.shape

True

In [9]:
X = torch.arange(9.0).reshape(3, 3)
K = torch.tensor([[0, 1], [2, 3]])
Y = d2l.corr2d(X, K)
Y

tensor([[19., 25.],
        [37., 43.]])

In [10]:
def kernel2matrix(K):
    k, W = torch.zeros(5), torch.zeros((4, 9))
    k[:2], k[3:5] = K[0, :], K[1, :]
    W[0, :5], W[1, 1:6], W[2, 3:8], W[3, 4:] = k, k, k, k
    return W

W = kernel2matrix(K)
W

tensor([[0., 1., 0., 2., 3., 0., 0., 0., 0.],
        [0., 0., 1., 0., 2., 3., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 2., 3., 0.],
        [0., 0., 0., 0., 0., 1., 0., 2., 3.]])

In [11]:
Y == torch.mv(W, X.reshape(-1)).reshape(2, 2)

tensor([[True, True],
        [True, True]])

In [12]:
X = torch.tensor([[0.0, 1], [2, 3]])
Y = trans_conv(X, K)
Y == torch.mv(W.T, X.reshape(-1)).reshape(3, 3)

tensor([[True, True, True],
        [True, True, True],
        [True, True, True]])